### 이진_다중 분류 모델_KNN
- 데이터 : iris.csv
- 3개 품종 중 1개 선택 => 예) Setosa 와 Others  : 2진분류 진행 
- 3개 품종 다중 분류 진행
- scale 전후 모델 성능 비교
- 최적 K 찾기

[1] 모듈 로딩 및 데이터 준비 <hr>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import koreanize_matplotlib

In [2]:
DATA_FILE = '../Data/iris.csv'

irisDF = pd.read_csv(DATA_FILE)